# This code is to use the Get_initial_cap to Get initial capacity to be Niall's 4.86Ah

In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing
import scipy.optimize

In [ ]:
# There maybe some problems with the choice Model option so just start from simplest code:

In [ ]:
Chemistry=pb.parameter_sets.OKane2022
Chemistry["electrolyte"] = "lipf6_Nyman2008"
Para_0=pb.ParameterValues(chemistry=Chemistry)
model_options={
    "calculate discharge energy":"true",
    "particle": "Fickian diffusion",          
    "SEI":"interstitial-diffusion limited",   
    "SEI on cracks":"true",  
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",      
    "particle mechanics":("swelling and cracking", "swelling only"), 
    "loss of active material":"stress-driven", 
    "lithium plating":"partially reversible"      }
Model_0 = pb.lithium_ion.DFN(options=model_options ) #
Neg1SOC_in = 0.57; Pos1SOC_in=0.42
c_Neg1SOC_in = (
    Para_0["Maximum concentration in negative electrode [mol.m-3]"]
    *Neg1SOC_in)
c_Pos1SOC_in = (
    Para_0["Maximum concentration in positive electrode [mol.m-3]"]
    *Pos1SOC_in)
Para_0.update(
    {"Initial concentration in negative electrode [mol.m-3]":
    c_Neg1SOC_in})
Para_0.update(
    {"Initial concentration in positive electrode [mol.m-3]":
    c_Pos1SOC_in})
V_max = 4.2;        
V_min = 2.5; 
exp_RPT_text = [ (
    f"Discharge at 0.1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.1C until {V_max} V" ) ] # (5 minute period)
RPT_Cycles=2
Experiment_RPT    = pb.Experiment( exp_RPT_text * RPT_Cycles     ) 
Sim_0    = pb.Simulation(
    Model_0,        experiment = Experiment_RPT,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),) #mode="safe"
Sol_0    = Sim_0.solve(calc_esoh=False)

In [ ]:
Chemistry=pb.parameter_sets.OKane2022 # Chen2020 # 
#Chemistry["negative electrode"]="graphite_OKane2022";
print(Chemistry)
Para_0=pb.ParameterValues(chemistry=Chemistry)
Para_0["Negative electrode cracking rate"]

In [4]:
Chemistry=pb.parameter_sets.OKane2022 # Chen2020 # 
#Chemistry["electrolyte"] = "lipf6_Nyman2008"
#Chemistry["negative electrode"] = "graphite_Chen2020"
#Chemistry["sei"] = "example"
Para_0=pb.ParameterValues(chemistry=Chemistry)
model_options={
    "calculate discharge energy":"true",
    "particle": "Fickian diffusion",          
    "SEI":"interstitial-diffusion limited",   
    #"SEI on cracks":"true",  
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",      
    #"particle mechanics":("swelling and cracking", "swelling only"), 
    "loss of active material":"stress-driven", 
    "lithium plating":"partially reversible"      
}
Model_0 = pb.lithium_ion.DFN(options=model_options ) #
""" Neg1SOC_in = 0.57; Pos1SOC_in=0.42
c_Neg1SOC_in = (
    Para_0["Maximum concentration in negative electrode [mol.m-3]"]
    *Neg1SOC_in)
c_Pos1SOC_in = (
    Para_0["Maximum concentration in positive electrode [mol.m-3]"]
    *Pos1SOC_in)
Para_0.update(
    {"Initial concentration in negative electrode [mol.m-3]":
    c_Neg1SOC_in})
Para_0.update(
    {"Initial concentration in positive electrode [mol.m-3]":
    c_Pos1SOC_in}) """
Para_0.update( {"Inner SEI reaction proportion":0.5})
Para_0.update( {"Initial inner SEI thickness [m]":2.5E-9})
Para_0.update( {"Initial outer SEI thickness [m]":2.5E-9})
Para_0.update( {"Ratio of lithium moles to SEI moles":2})
var = pb.standard_spatial_vars  
var_pts = {
    var.x_n: 20,  
    var.x_s: 10,  
    var.x_p: 20,  
    var.r_n: int(200),  
    var.r_p: int(200),  }       
submesh_types = Model_0.default_submesh_types
particle_mesh = pb.MeshGenerator(
    pb.Exponential1DSubMesh, 
    submesh_params={"side": "right", "stretch": 1.5})
submesh_types["negative particle"] = particle_mesh
submesh_types["positive particle"] = particle_mesh 
V_max = 4.2;        
V_min = 2.5; 
exp_RPT_text = [ (
    f"Discharge at 0.1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.1C until {V_max} V" ) ] # (5 minute period)
exp_RPT_s = [ (
    f"Discharge at 0.1C for 10 s",  ) ] 
RPT_Cycles=1
Experiment_RPT    = pb.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_s    = pb.Experiment( exp_RPT_s * RPT_Cycles     ) 
Sim_0    = pb.Simulation(
    Model_0,        experiment = Experiment_s,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    submesh_types=submesh_types) #mode="safe"
Sol_0    = Sim_0.solve(calc_esoh=False)

In [ ]:
# Quick plot
label = ["Sol_0"] 
output_variables3 = [
    "Terminal voltage [V]",
    "Negative electrode capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Loss of capacity to lithium plating [A.h]",
    "Loss of capacity to SEI [A.h]",
    "Loss of capacity to SEI on cracks [A.h]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    
]
quick_plot = pb.QuickPlot([Sol_0], output_variables3,label,variable_limits='fixed') #     
quick_plot.dynamic_plot();

In [ ]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2_Crack_simp import (
    GetScan,
    recursive_scan,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,
    Get_initial_cap,
    Get_initial_cap2
)

In [ ]:
########################  Input  ########################
""" 12 
[4.8728428390998655, 4.858536253367931] 
[0.8519999999999996, 0.8471043791226032] 
[0.2700000000000001, 0.2710723194914279]"""

# all values here must be a list, even it is a single object
Para_dict_All = {
   "Total ageing cycles":[1170,],
   "Ageing cycles between RPT":[78,],
   "Update cycles for ageing":[26,],
   "Cycles within RPT":[2,],
   "Ageing temperature":[40,],
   "RPT temperature":[25,],
   "Particle mesh points":[30,],   # Simon uses 30
   #"Exponential mesh stretch":[1.0,],
   "Para_Set":[ "OKane2022",],
   "electrolyte":["lipf6_Nyman2008"],
   "Model option":[
         {
            "calculate discharge energy":"true",
            "particle": "Fickian diffusion",          
            "SEI":"interstitial-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   "Initial Neg SOC":[0.850],    #list(np.linspace(0.84,0.90,6)),
   "Initial Pos SOC":[0.270], # list(np.linspace(0.22,0.27,6)),

   # Solvent consumption sub-model
   "Initial electrolyte excessive amount ratio":[1.05,], # set to <1 for DryOut=Off 
   "Current solvent concentration in the reservoir [mol.m-3]":[4541.0,],
   "Current electrolyte concentration in the reservoir [mol.m-3]":[1000,],
   "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":[1.0,],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],
   #"Negative electrode diffusivity [m2.s-1]":[2e-13,], # default: a function, about 3.3e-14
   #"Positive electrode diffusivity [m2.s-1]":[1e-13,], # default: 4e-15
   
   # general SEI
   'SEI resistivity [Ohm.m]':[2E5,],
   # solvent-diffusion limited
   'Outer SEI solvent diffusivity [m2.s-1]':[2.5e-22],       # default: 2.5e-22
   'Bulk solvent concentration [mol.m-3]':[4541.0,],          # default: 4541.0
   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-24,],    
   'Lithium interstitial reference concentration [mol.m-3]':[5,],
   # ec reaction limited
   'EC diffusivity [m2.s-1]':[2e-24,],
   'SEI kinetic rate constant [m.s-1]':[1e-21,], 
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-6,],            # default: 1e-6
   'Lithium plating kinetic rate constant [m.s-1]':[1E-9,], # default: 1e-9
   # Crack model
   "Negative electrode cracking rate":[ 3.9e-23,],   # default: function, ~3.9e-20
   "Positive electrode cracking rate":[ 3.9e-23,],   # default: function, ~3.9e-20
   "Negative electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})


In [ ]:

Sol_0,Cap, Neg1SOC, Pos1SOC = Get_initial_cap2( Para_dict_list[0])

""" 12 [4.8728428390998655, 4.858536253367931] 
[0.8519999999999996, 0.8471043791226032] 
[0.2700000000000001, 0.2710723194914279] """

In [ ]:
Sol_1,Cap_1, Neg1SOC_1, Pos1SOC_1 = Get_initial_cap( 
    Para_dict_list[0],0.8475919170851502, 0.27107950262336467)

In [ ]:
print(Cap, Neg1SOC, Pos1SOC)


In [ ]:
print(Cap_1, Neg1SOC_1, Pos1SOC_1)

In [ ]:
# Quick plot
label = ["Sol_0","Sol_1"] 
output_variables3 = [
    "Negative electrode capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Loss of capacity to lithium plating [A.h]",
    "Loss of capacity to SEI [A.h]",
    "Loss of capacity to SEI on cracks [A.h]",
    
]
quick_plot = pb.QuickPlot([Sol_0,Sol_1], output_variables3,label,variable_limits='fixed') #     
quick_plot.dynamic_plot();

In [ ]:
print(Sol_0.cycles[1].steps[0]["Negative electrode SOC"].entries[0])
print(Sol_0.cycles[1]["Negative electrode SOC"].entries[0])
print(Sol_0.cycles[1].steps[0]["Positive electrode SOC"].entries[0])
print(Sol_0.cycles[1]["Positive electrode SOC"].entries[0])
